In [10]:
import pandas as pd
import nltk
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') 
from TweetScraper.TweetScraper.pipelines import TweetscraperPipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
#initiation variables
tweet_db_loc="../db_files/tweet_data.db"
spider=None 
tweet_con=TweetscraperPipeline(string=tweet_db_loc)
tweet_con.open_spider(spider=spider)
data=pd.DataFrame(data=tweet_con.get_tweets(), columns=["tweet_text","tweet_type"])
tweet_con.close_spider(spider=spider)
len(data), len(data[data["tweet_type"]==1]), len(data[data["tweet_type"]==0])


2020-02-16 13:19:20,755 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-16 13:19:20,756 INFO sqlalchemy.engine.base.Engine ()
2020-02-16 13:19:20,758 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-16 13:19:20,760 INFO sqlalchemy.engine.base.Engine ()
2020-02-16 13:19:20,761 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("tweet_data")
2020-02-16 13:19:20,762 INFO sqlalchemy.engine.base.Engine ()
2020-02-16 13:19:20,767 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-16 13:19:20,770 INFO sqlalchemy.engine.base.Engine SELECT tweet_data.id AS tweet_data_id, tweet_data.tweet AS tweet_data_tweet, tweet_data.tweet_type AS tweet_data_tweet_type, tweet_data.tweet_loc AS tweet_data_tweet_loc 
FROM tweet_data ORDER BY tweet_data.id
2020-02-16 13:19:20,771 INFO sqlalchemy.engine.base.Engine ()
2020-02-16 13:19:21,534 INFO sqlalchemy.engine.base.Engine ROLLBACK


(26820, 20620, 6200)

In [13]:
def get_word_ls(data_frame):
    tweet_series=data_frame["tweet_text"]
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    remove_punctuations = re.compile("\W+")
    excluded_words=stopwords.words("english")
    sentences_tokens=list()
    for idx, item in enumerate(tweet_series):
        item = item.lower()
        item=remove_punctuations.sub(" ", item)
        word_token=tokenizer.tokenize(item)
        word_token=[w for w in word_token if w not in excluded_words]
        lemmed = [WordNetLemmatizer().lemmatize(w) for w in word_token]
        sentences_tokens.append(lemmed)
    return sentences_tokens

In [15]:
def get_bag_words(sentences_tokens):
    bag_words=set()
    for sentences in sentences_tokens:
        bag_words.update(sentences)
    bag_words=list(bag_words)
    return bag_words

def word_index (list_, word):
    return list_.index(word)

def get_one_hot_encode (sentences,word_bag):
    #sentences must be a python list
    one_hot_encode=list()
    for line in sentences:
        vect=[0 for i in range (0,len(word_bag))]
        for word in line :
            if word in bag_words :
                vect[word_index(word_bag,word)]=1
        one_hot_encode.append(vect)
    return one_hot_encode
        

In [17]:
dem_tokens=get_word_ls(data)
bag_words=get_bag_words(dem_tokens)
oneHotEncode_frame=pd.DataFrame(data=get_one_hot_encode(dem_tokens,bag_words), columns= bag_words)
oneHotEncode_frame

MemoryError: Unable to allocate 7.02 GiB for an array with shape (26820, 35136) and data type object

In [158]:
data["tweet_type"]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [160]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
n=6
X_train=oneHotEncode_frame[:n]
y_train=y[:n]
X_test=oneHotEncode_frame[n:]
y_test=y[n:]

y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [161]:
print(y_pred)
print(y_test)

[1 1 1 1]
[1, 1, 1, 1]


2020-02-16 13:18:52,022 INFO sqlalchemy.engine.base.Engine ROLLBACK
